In [24]:
# Importar librerias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#Me interesa: ; dealer_up; initial_hand; actions_taken
columnas = ['dealer_up','initial_hand','actions_taken']
df = pd.read_csv("csv_reducido",usecols=columnas)

df.head(10)

,dealer_up,initial_hand,actions_taken
0,10,"[6, 3]","[['H', 'H']]"
1,10,"[10, 5]",[['R']]
2,2,"[6, 8]",[['S']]
3,8,"[7, 3]",[['D']]
4,6,"[10, 5]",[['S']]
5,11,"[2, 7]","[['N', 'H', 'S']]"
6,9,"[6, 10]",[['R']]
7,10,"[10, 10]",[[]]
8,6,"[10, 2]",[['S']]
9,10,"[5, 10]",[[]]


In [ ]:
# Sumar toda la initial hand para que sea un valor
df['hand_value'] = df['initial_hand'].apply(lambda x: sum(eval(x)) if isinstance(x, str) else sum(x))

# Solo me interesa la primera accion, si Hitteo, o si hizo Stay
acciones_validas = ['H','S']

# Recorro la columna actions taken
def get_accion_valida(actions):
    try:
        actions = eval(actions)[0] if isinstance(actions, str) else actions[0]
    except Exception:
        return None
    for action in actions:
        if action in acciones_validas:
            return action
    return None


df['first_action'] = df['actions_taken'].apply(get_accion_valida)
df = df[df['first_action'].notnull()]
df['action'] = df['first_action'].map({'S':0,'H':1})

inputs = df[['dealer_up','hand_value']]
outputs = df[['action']]
df_preview = pd.concat([inputs, outputs], axis=1)
print(df_preview.head())


   dealer_up  hand_value  action
0         10           9       1
2          2          14       0
4          6          15       0
5         11           9       1
8          6          12       0
